In [11]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import os, sys
sys.path.append('../../src/')

import toolbox

In [12]:
# List 
# individual coins: btc, ada, eth, ltc, xrp
# indexes: BITX, BITW20, BITW100, CRIX

index = ['BITX', 'BITW20', 'BITW70', 'BITW100', 'CRIX']
coin_arr = ['ada', 'eth', 'ltc', 'xrp']

In [13]:
# Table 1a: Moments, dependency with future, start and end
def summary_statistics(df, coin_name, start='2000-01-01'):
    
    df = df.loc[df.Date >= start,:]
    
    rs = df.loc[:,'log return %s'%coin_name]
    rf = df.loc[:, 'log return future']
    
    m1 = np.mean(rs)
    m2 = np.std(rs)
    m3 = ss.skew(rs) # Pearson coefficient of skewness
    m4 = ss.kurtosis(rs) # Pearson coefficient of kurtosis

    _max = np.max(rs)
    _min = np.min(rs)

    tau = ss.kendalltau(rs,rf)[0]
    rho = np.corrcoef(rs, rf)[0,1]

    end = df.Date.iloc[0]
    start = df.Date.iloc[-1]
    
    _dict = {'m1':m1, 'm2':m2, 'm3':m3, 'm4':m4, 
            'max':_max, 'min':_min, 
            'tau':tau, 'rho':rho, 
            'start':start[:10], 'end':end[:10]}
    
    return pd.DataFrame([_dict], [coin_name])

In [14]:
coin_arr = ['ada', 'eth', 'ltc', 'xrp']
table1 = pd.DataFrame()
for c in coin_arr:
    coin_name = c
    file_name = 'BBT_future_Tiingo_'+coin_name+'.csv'
    path = '../../data/cleaned_data/'+file_name
    df = pd.read_csv(path)
    df = df.sort_values('Date', ascending=False)
    table1 = table1.append(summary_statistics(df, c, '2018-08-01'))
    

In [15]:
# btc
path = '../../data/cleaned_data/BBT_Tiingo.csv'
df = pd.read_csv(path)

table1 = table1.append(summary_statistics(df, 'bitcoin', '2018-08-01'))
table1 = table1.append(summary_statistics(df, 'future', '2018-08-01'))

In [16]:
for index_name in index:
    file_name = 'BBT_future_'+index_name+'.csv'
    path = '../../data/cleaned_data/'+file_name
    df = pd.read_csv(path)
    df = df.sort_values('Date', ascending=False)
    table1 = table1.append(summary_statistics(df, index_name, '2018-08-01'))

In [17]:
_index = ['btc','eth','ada','ltc','xrp','BITX','CRIX','BITW100','BITW20','BITW70']

# table1

In [18]:
def risk_statistics(df, coin_name, start='2000-01-01'):
    
    df = df.loc[df.Date >= start,:]

    rs = df.loc[:,'log return %s'%coin_name]
    ERM = toolbox.ERM_estimate_trapezoidal(10, rs)
    ES1 = toolbox.ES(0.01, rs)
    ES5 = toolbox.ES(0.05, rs)
    VaR1 = toolbox.VaR(0.01, rs)
    VaR5 = toolbox.VaR(0.05, rs)
    Var = toolbox.Variance(rs)
    
    _dict = {'ERM':ERM, 'ES1':ES1, 'ES5':ES5, 'VaR1':VaR1, 
             'VaR5':VaR5, 'Var':Var}
    
    return pd.DataFrame([_dict], [coin_name])

In [19]:
table2 = pd.DataFrame()

for c in coin_arr:
    coin_name = c
    file_name = 'BBT_future_Tiingo_'+coin_name+'.csv'
    path = '../../data/cleaned_data/'+file_name
    df = pd.read_csv(path)
    df = df.sort_values('Date', ascending=False)
    table2 = table2.append(risk_statistics(df, c, '2018-08-01'))
    
path = '../../data/cleaned_data/BBT_Tiingo.csv'
df = pd.read_csv(path)

table2 = table2.append(risk_statistics(df, 'bitcoin'))
table2 = table2.append(risk_statistics(df, 'future'))

for index_name in index:
    file_name = 'BBT_future_'+index_name+'.csv'
    path = '../../data/cleaned_data/'+file_name
    df = pd.read_csv(path)
    df = df.sort_values('Date', ascending=False)
    table2 = table2.append(risk_statistics(df, index_name, '2018-08-01'))

In [20]:
# table2 = table2.reindex(_index)
table2

,ERM,ES1,ES5,VaR1,VaR5,Var
ada,0.095791,0.214140,0.152781,0.170588,0.101729,0.004419
eth,0.088572,0.217855,0.144642,0.169135,0.097715,0.003672
ltc,0.096585,0.242580,0.158273,0.185884,0.101919,0.004139
xrp,0.095668,0.259285,0.159456,0.171372,0.104897,0.005142
bitcoin,0.073294,0.190733,0.120182,0.135245,0.083006,0.002257
future,0.075553,0.191951,0.123777,0.149558,0.082433,0.002391
BITX,0.072262,0.180512,0.121851,0.147969,0.076431,0.002242
BITW20,0.087091,0.222895,0.142478,0.175095,0.083903,0.003024
BITW70,0.094595,0.239928,0.156907,0.198197,0.098095,0.003424
BITW100,0.073052,0.184992,0.123148,0.146904,0.077594,0.002270


In [21]:
_index = ['future','bitcoin','eth','ada','ltc','xrp','BITX','CRIX','BITW100','BITW20','BITW70']

In [22]:
table1 = table1.reindex(_index)
table2 = table2.reindex(_index)

In [23]:
print(table1.iloc[:,:-2].to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &        m1 &        m2 &        m3 &         m4 &       max &       min &       tau &       rho \\
\midrule
future  &  0.002266 &  0.047580 & -0.366160 &   4.792542 &  0.222514 & -0.269920 &  1.000000 &  1.000000 \\
bitcoin &  0.002261 &  0.045790 & -0.387484 &   5.044245 &  0.211384 & -0.259965 &  0.921749 &  0.992384 \\
eth     &  0.002594 &  0.060593 & -0.069137 &   4.046130 &  0.315242 & -0.320144 &  0.613693 &  0.786019 \\
ada     &  0.003486 &  0.066478 &  0.007250 &   2.015130 &  0.258805 & -0.268528 &  0.503697 &  0.685398 \\
ltc     &  0.001298 &  0.064339 & -0.333109 &   4.162287 &  0.272965 & -0.375913 &  0.590592 &  0.779309 \\
xrp     &  0.001138 &  0.071705 &  0.683356 &  12.436338 &  0.530037 & -0.527652 &  0.493555 &  0.506819 \\
BITX    &  0.002092 &  0.047350 & -0.552960 &   4.038681 &  0.196783 & -0.270220 &  0.865932 &  0.976451 \\
CRIX    &  0.002209 &  0.047011 & -0.494949 &   4.155185 &  0.194555 & -0.271385 &  0.849397 &  

In [24]:
table2

,ERM,ES1,ES5,VaR1,VaR5,Var
future,0.075553,0.191951,0.123777,0.149558,0.082433,0.002391
bitcoin,0.073294,0.190733,0.120182,0.135245,0.083006,0.002257
eth,0.088572,0.217855,0.144642,0.169135,0.097715,0.003672
ada,0.095791,0.214140,0.152781,0.170588,0.101729,0.004419
ltc,0.096585,0.242580,0.158273,0.185884,0.101919,0.004139
xrp,0.095668,0.259285,0.159456,0.171372,0.104897,0.005142
BITX,0.072262,0.180512,0.121851,0.147969,0.076431,0.002242
CRIX,0.070835,0.181904,0.118451,0.144185,0.074766,0.002210
BITW100,0.073052,0.184992,0.123148,0.146904,0.077594,0.002270
BITW20,0.087091,0.222895,0.142478,0.175095,0.083903,0.003024


In [25]:
np.log(300+4)

5.717027701406222

In [26]:
np.log(300+2)

5.71042701737487

In [27]:
import copulae1 as C
import KDEs as K

In [28]:
kde_rs = K.KDE(rs, 'Gaussian')
kde_rf = K.KDE(rf, 'Gaussian')

C1 = C.Gaussian({'rho':0.9}, kde_rs, kde_rf)
C2 = C.t_Copula({'rho':0.9, 'nu':4}, kde_rs, kde_rf, 2)

NameError: name 'rs' is not defined

In [29]:
u = kde_rs.cdf(rs)
v = kde_rf.cdf(rf)
C1.canonical_calibrate(u,v)
l = np.array([np.log(C1.c(u[i],v[i])) for i in range(len(rs))])
np.sum(l[np.isfinite(l)])

NameError: name 'kde_rs' is not defined

In [252]:
u = kde_rs.cdf(rs)
v = kde_rf.cdf(rf)
C2.canonical_calibrate(u,v)
l = np.array([np.log(C2.c(u[i],v[i])) for i in range(len(rs))])
np.sum(l[np.isfinite(l)])

Optimization terminated successfully.
         Current function value: -0.402058
         Iterations: 75
         Function evaluations: 158


326.9511478845575

In [253]:
0.618854*300

185.6562